# Evaluate 100 alignments

## 1 Setup

Flags

In [1]:
N_TEST = 100

Setup the database

In [2]:
import os, sys
sys.path.insert(1, os.path.abspath('../..'))
import analysis

FIG = os.path.join(os.path.abspath(os.path.curdir), '{}.png')
DB_NAME = 'spreadr_' + os.path.split(os.path.abspath(os.path.curdir))[1]
analysis.setup(DB_NAME)
print('Database:', DB_NAME)

HAND_ALIGNMENTS_FILE = '../../data/alignments/' + DB_NAME + '/sebastien-lerique.csv'
CODINGS_ALIGNMENTS_FILE = ('../../data/codings/'
                           + DB_NAME
                           + '/alignments'
                           + '/sebastien-lerique_on_sebastien-lerique-deep-alignments.csv')

Database: spreadr_exp_3


Imports for the analysis

In [3]:
import csv
import random
from collections import Counter

from gists.models import Sentence

from analysis.transformations import format_deep_alignments

## 2 Look at 100 non-trivial deep alignments to code their quality

Only look at sentences that have a non-null ordered word distance, to make sure there is change to align on.

In [1]:
def find_sentences(sentence_ids, n):
    found = []
    while len(found) < n:
        sentence_id = sentence_ids.pop()
        sentence = Sentence.objects.get(id=sentence_id)
        if sentence.parent.ow_distance(sentence) == 0:
            # Skip this sentence, it has no interesting transformations
            continue
        found.append(sentence_id)
    return found

In [4]:
hand_aligned_ids = []
with open(HAND_ALIGNMENTS_FILE, 'r') as hand_alignments_file:
    reader = csv.DictReader(hand_alignments_file)
    for row in reader:
        hand_aligned_ids.append(row['sentence_id'])

In [10]:
kept_sentence_ids = list(Sentence.objects.kept
                         .filter(parent__isnull=False)
                         .values_list('id', flat=True))
testable_sentence_ids = list(set(kept_sentence_ids).difference(hand_aligned_ids))
random.shuffle(testable_sentence_ids)

found_sentences = find_sentences(testable_sentence_ids, N_TEST)
found_sentences

[2646,
 1636,
 3415,
 359,
 464,
 430,
 2179,
 2965,
 660,
 411,
 356,
 86,
 2155,
 516,
 1961,
 456,
 3049,
 655,
 2680,
 1147,
 931,
 2310,
 1693,
 886,
 2380,
 937,
 2402,
 840,
 3034,
 1330,
 3340,
 1569,
 2647,
 2187,
 144,
 1340,
 2069,
 1266,
 2579,
 1372,
 3273,
 2664,
 2196,
 794,
 2630,
 475,
 2807,
 1337,
 1966,
 2095,
 1246,
 2189,
 3977,
 1849,
 1177,
 322,
 267,
 227,
 2153,
 2577,
 2581,
 3341,
 1582,
 256,
 3371,
 567,
 2700,
 204,
 2363,
 1748,
 101,
 980,
 2679,
 2765,
 2278,
 65,
 3480,
 583,
 3625,
 3197,
 859,
 2504,
 3099,
 514,
 1233,
 915,
 1493,
 3683,
 3298,
 722,
 1352,
 2199,
 3651,
 521,
 2109,
 3923,
 304,
 670,
 335,
 3110]

## 3 Hand-code the alignments for number of errors

This is done by hand, and the result is in `CODINGS_ALIGNMENTS_FILE`.

I coded alignments with the number of words that we would have to change to get an optimal alignment. Note that for words split into two or merged, no errors were counted if it was coded as all insertions+deletions, or if the main meaning bearers were matched.

The feeling is that, often, errors are words at the margin of bigger changes, and the overall operation by the subject is nonetheless well captured.

## 4 Counting the errors

In [4]:
n_errors = []
with open(CODINGS_ALIGNMENTS_FILE, 'r') as codings_file:
    reader = csv.DictReader(codings_file)
    for row in reader:
        n_errors.append(row['n_errors'])

In [8]:
for (error_count, count) in Counter(n_errors).items():\
    print('{} sentences have {} error(s)'.format(count, error_count))

79 sentences have 0 error(s)
12 sentences have 1 error(s)
4 sentences have 2 error(s)
1 sentences have 4 error(s)
1 sentences have 6 error(s)
2 sentences have 3 error(s)
1 sentences have 5 error(s)
